<a href="https://colab.research.google.com/github/yagyeshgoyal/Bhopal-LULC/blob/main/Bhopallulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np

In [9]:
a = 1;
b = 1;
print(a+b);

2
